In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

In [16]:
from model.EPInformer import EPInformer_v2
from scripts.utils import prepare_input, FastaStringExtractor

In [9]:
flowfish_df = pd.read_excel('./data/CRISPRi-FlowFISH_Fulco2019/41588_2019_538_MOESM3_ESM.xlsx',sheet_name='Supplementary Table 6a', skiprows=[0])
# ensid_gene = pd.read_csv('./data/CRISPRi-FlowFISH_Fulco2019/GeneName2ENSID.csv')
RNA_feats = pd.read_csv('./data/GM12878_K562_18377_gene_expr_fromXpresso.csv', index_col='Gene stable ID')[['Gene name', 'UTR5LEN_log10zscore','CDSLEN_log10zscore','INTRONLEN_log10zscore','UTR3LEN_log10zscore','UTR5GC','CDSGC','UTR3GC', 'ORFEXONDENSITY_log10zscore']]
ensid_gene = RNA_feats[['Gene name']].reset_index()
flowfish_df['significant_enhancer'] = flowfish_df['Significant']&(flowfish_df['Fraction change in gene expr'] < 0)
flowfish_df = flowfish_df.merge(ensid_gene, left_on='Gene', right_on='Gene name', how='left')
flowfish_df['Xpresso_gene'] = False
intersec_ensid = list(set(flowfish_df['Gene stable ID']).intersection(set(RNA_feats.index)))
flowfish_df = flowfish_df.set_index('Gene stable ID')
flowfish_df.loc[intersec_ensid, 'Xpresso_gene'] = True
flowfish_df = flowfish_df[flowfish_df['Xpresso_gene']].reset_index()
flowfish_df['mid'] = flowfish_df['start'] + (flowfish_df['end'] - flowfish_df['start'])/2
flowfish_df['Distance'] = np.abs(flowfish_df['mid'] - flowfish_df['Gene TSS'])
flowfish_100kb_df = flowfish_df[flowfish_df['Distance'] <= 100_000]
flowfish_100kb_df['#true_enhance_by_gene'] = flowfish_100kb_df.groupby('Gene')['significant_enhancer'].transform('sum') #  = 

In [13]:
expr_df = pd.read_csv('./data/GM12878_K562_18377_gene_expr_fromXpresso.csv', index_col='ENSID')[['K562_CAGE_128*3_sum', 'Actual_K562', 'Gene name']]
expr_flowfish_100kb_df = expr_df.merge(flowfish_100kb_df, left_index=True, right_on='Gene stable ID', how='right').reset_index()

In [49]:
gene_list = flowfish_100kb_df['Gene name'].dropna().unique()

In [15]:
promoter_df = pd.read_csv('./data/CRISPRi-FlowFISH_Fulco2019/DNase_ENCFF257HEE_Neighborhoods/GeneList.txt', sep='\t', index_col='symbol')
promoter_df['PromoterActivity'] = np.sqrt(promoter_df['H3K27ac.RPM.TSS1Kb']*promoter_df['DHS.RPM.TSS1Kb'])

In [17]:
hg19_fasta_extractor = FastaStringExtractor("../hg19.fa")